# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

df = pd.read_csv('ToothGrowth.csv')
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


In [2]:
df.supp.value_counts()

OJ    30
VC    30
Name: supp, dtype: int64

In [14]:
df.dose.value_counts()

2.0    20
1.0    20
0.5    20
Name: dose, dtype: int64

## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [15]:
formula = 'len ~ C(supp) + C(dose)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
table

,sum_sq,df,F,PR(>F)
C(supp),205.350000,1.0,14.016638,4.292793e-04
C(dose),2426.434333,2.0,82.810935,1.871163e-17
Residual,820.425000,56.0,NaN,NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

In [ ]:
# Your comment here
# It appears that both features are influential, with the dosage being the most significant.

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [9]:
# Your code here
vc = df.loc[df.supp == 'VC']['len']
oj = df.loc[df.supp == 'OJ']['len']

Now run a t-test between these two groups and print the associated two-sided p-value: 

In [17]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
import scipy.stats as stats
results = stats.ttest_ind(vc, oj, equal_var = False)
results

Ttest_indResult(statistic=-1.91526826869527, pvalue=0.06063450788093387)

## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [18]:
# Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
formula = 'len ~ C(supp)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
table
# Compare the p-value to that of the t-test above. 
# They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)

,sum_sq,df,F,PR(>F)
C(supp),205.350000,1.0,3.668253,0.060393
Residual,3246.859333,58.0,NaN,NaN


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [22]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [24]:
# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
# for all combinations of the supplement-dose groups listed above. 
pvals = {}
for group1 in df.groupby(['supp', 'dose'])['len']:
    pvals[group1[0]] = {}
    for group2 in df.groupby(['supp', 'dose'])['len']:
        pvals[group1[0]][group2[0]] = stats.ttest_ind(group1[1], group2[1], equal_var = False)[1]
    
pvals_df = pd.DataFrame.from_dict(pvals)
pvals_df
# (Since there isn't a control group, compare each group to every other group.)

OJ                                        VC                \
             0.5           1.0           2.0           0.5           1.0   
OJ 0.5  1.000000  8.784919e-05  1.323784e-06  6.358607e-03  4.601033e-02   
   1.0  0.000088  1.000000e+00  3.919514e-02  3.655207e-08  1.038376e-03   
   2.0  0.000001  3.919514e-02  1.000000e+00  1.362140e-11  2.361074e-07   
VC 0.5  0.006359  3.655207e-08  1.362140e-11  1.000000e+00  6.811018e-07   
   1.0  0.046010  1.038376e-03  2.361074e-07  6.811018e-07  1.000000e+00   
   2.0  0.000007  9.652612e-02  9.638516e-01  4.681577e-08  9.155603e-05   

                      
                 2.0  
OJ 0.5  7.196254e-06  
   1.0  9.652612e-02  
   2.0  9.638516e-01  
VC 0.5  4.681577e-08  
   1.0  9.155603e-05  
   2.0  1.000000e+00

## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.